# Дисперсионный анализ

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

import scipy.stats as st
import seaborn as sns

import statsmodels.api as sm

from statsmodels.formula.api import ols

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

from statsmodels.stats.weightstats  import ttest_ind

## Лечение гипертонии
**Maxwell S.E., Delaney H.D. Designing Experiments and Analyzing Data: A Model Comparison Perspective. (1990). Глава 8, стр. 325.**

72 пациента проходили лечение от гипертонии. Для лечения использовались три вида лекарств, при этом их эффект изучался как при использовании специальной диеты, так и в её отсутствии; кроме того, в половине случаев применялась психотерапия. Данные — артериальное давление пациента по окончании лечения.

Требуется сравнить эффективность методов лечения гипертонии.

### Чтение данных

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem6/data/hyper.txt', delimiter='\t')
data.head()

In [ ]:
data.sample(10, random_state=42)

In [ ]:
data.describe(include='all')

### Трехфакторный дисперсионный анализ для пациентов

In [ ]:
lm = ols('pressure ~ C(biofeedback) * C(diet) * C(drug)', data=data).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

In [ ]:
table[table['PR(>F)'] < 0.05]

#### Построим графики взаимодействий

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

In [ ]:
plt.title('Diet: absent')
_ = sns.pointplot(x="biofeedback", y="pressure", data=data.loc[data['diet']=='absent'], hue='drug')
plt.show()

plt.title('Diet: present')
_ = sns.pointplot(x="biofeedback", y="pressure", data=data.loc[data['diet']=='present'], hue='drug')
plt.show()

In [ ]:
plt.title('biofeedback: absent')
_ = sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='absent'], hue='drug')
plt.show()

plt.title('biofeedback: present')
_ = sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='drug')
plt.show()

Тройное взаимодействие значимо, поэтому разделим выборку по одному из признаков.

### Двухфакторный дисперсионный анализ для пациентов, проходящих психотерапию

In [ ]:
lm = ols('pressure ~ C(diet) * C(drug)', data=data.loc[data['biofeedback']=='present']).fit()
table = sm.stats.anova_lm(lm)
table

Есть значимое межфакторное взаимодействие.

#### Графики

In [ ]:
_ = sns.pointplot(x="diet", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='drug')
plt.show()
_ = sns.pointplot(x="drug", y="pressure", data=data.loc[data['biofeedback']=='present'], hue='diet')
plt.show()

Поэтому снова поделим выборку.

### Однофакторный анализ для пациентов, проходящих психотерапию и сидящих на диете:

In [ ]:
_ = sns.boxplot(x='drug', y='pressure', data=data[(data['biofeedback']=='present') & (data['diet']=='present')])
plt.show()

In [ ]:
lm =  ols('pressure ~ C(drug)',data=data.loc[(data['biofeedback']=='present') & (data['diet']=='present')]).fit()
table = sm.stats.anova_lm(lm)
table

Ожидаемый p-value: ```~0.850126```

### Однофакторный анализ для пациентов, проходящих психотерапию и не сидящих на диете:

In [ ]:
_ = sns.boxplot(x='drug', y='pressure', data=data[(data['biofeedback']=='present') & (data['diet']=='absent')])

In [ ]:
lm =  ols('pressure ~ C(drug)',data=data.loc[(data['biofeedback']=='present') & (data['diet']=='absent')]).fit()
table = sm.stats.anova_lm(lm)
table

Ожидаемый p-value: ```~0.000269```

#### Исследуем различия в воздействии между группами

In [ ]:
mc = MultiComparison(data.loc[(data['biofeedback']=='present') & (data['diet']=='absent')]['pressure'],
                     data.loc[(data['biofeedback']=='present') & (data['diet']=='absent')]['drug'])
result = mc.tukeyhsd()

print(result)

Препарат 1 значимо эффективнее двух других.

### Двухфакторный дисперсионный анализ для пациентов, не проходящих психотерапию:

In [ ]:
lm = ols('pressure ~ C(diet) * C(drug)',data=data[data['biofeedback']=='absent']).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

Межфакторное взаимодействие незначимо:

In [ ]:
_ = sns.pointplot(x="diet", y="pressure", data=data[data['biofeedback']=='absent'], hue='drug')
plt.show()

_ = sns.pointplot(x="drug", y="pressure", data=data[data['biofeedback']=='absent'], hue='diet')
plt.show()

#### Исследуем попарные различия в воздействии каждого из лекарств в зависимости от диеты

In [ ]:
subset = data[data.diet == 'absent']
mc = MultiComparison(subset['pressure'], subset['drug'])
result = mc.tukeyhsd()
print(result)

In [ ]:
subset = data[data.diet == 'present']
mc = MultiComparison(subset['pressure'], subset['drug'])
result = mc.tukeyhsd()
print(result)

### Итого:

* если пациент не проходит психотерапию, соблюдение диеты приводит к значимому уменьшению давления при приёме любого из трёх препаратов;
* эффективность всех трёх препаратов неразличима для пациентов, сидящих на диете и проходящих психотерапию;
* для пациентов, проходящих психотерапию и не сидящих на диете, препарат 1 эффективнее в среднем примерно на 20 мм. рт. ст.
* Чтобы сделать другие возможные выводы (например, оценить эффекты психотерапии и эффект диеты для тех, кто проходит психотерапию), нужно разбить трёхфакторный анализ по двум другим признакам.


## Рост певцов хора
**Cleveland W. S. (1993) Visualizing Data. Summit, New Jersey: Hobart Press.**

В 1979 году было опрошено 235 членов нью-йоркской ассоциации хорового пения, для каждого из них известен рост и регистр голоса. Меняется ли средний рост вместе с регистром?



In [ ]:
data =  sm.datasets.get_rdataset('singer', 'lattice').data
data.head()

Переименуем 2 колонку и переведем рост в сантиметры

In [ ]:
data.columns = ['height', 'voice']
data['height'] *= 2.54

In [ ]:
data.describe()

In [ ]:
_ = sns.boxplot(x='height', y='voice', data=data)
plt.show()

### Однофакторный дисперсионный анализ:

In [ ]:
lm = ols('height ~ C(voice)', data=data).fit()
table = sm.stats.anova_lm(lm)
table

Ожидаемое p-value ~ `5.6e-40`

In [ ]:
data.groupby('voice').mean()

### Критерий Краскела-Уоллиса о равенстве средних:

In [ ]:
st.kruskal(*data.groupby('voice').groups.values())

### Критерий Бартлетта о равенстве дисперсий:

In [ ]:
st.bartlett(*data.groupby('voice').groups.values())

### HSD Тьюки:

#### Существенность различий между группами

In [ ]:
mc = MultiComparison(data['height'], data['voice'])
result = mc.tukeyhsd()
 
print(result)

### Итог:

- в целом, тип голоса существенно влияет на рост
- различия в этом влиянии присутствуют лишь между некоторыми парами голосов

## Влияние витамина C на рост зубов морских свинок
**Crampton E. W. (1947) The growth of the odontoblast of the incisor teeth as a criterion of vitamin C intake of the guinea pig. The Journal of Nutrition 33(5): 491–504.**

В эксперименте изучалось влияние витамина C на рост [одонтобластов](https://en.wikipedia.org/wiki/Odontoblast) морских свинок.

Рассматривалось три дозы витамина (0.5, 1 и 2 мг) и два способа его приёма (апельсиновый сок и в чистом виде (== аскорбиновая кислота)). В каждой из 6 групп по уровням двух факторов были произведены измерения для 10 морских свинок.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem6/data/ToothGrowth.csv', index_col=0)
data.head()

In [ ]:
data.supp.unique()

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=data)
plt.show()

_ = sns.boxplot(x='supp', y='len', data=data)
plt.show()

### Двухфакторный дисперсионный анализ

#### Без учета межфакторного взаимодействия:

In [ ]:
lm = ols('len ~ C(supp) + C(dose)',data=data).fit()
table = sm.stats.anova_lm(lm)
table

#### С учетом межфакторного взаимодействия:

In [ ]:
lm = ols('len ~ C(supp) * C(dose)', data=data).fit()
table = sm.stats.anova_lm(lm, typ=1)
table

### Межфакторное взаимодействие:

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

`sns.pointplot`: 
- точки -- средние значения исследуемого параметра по каждой из групп (значению категориальной переменной)
- 95%-й доверительный интервал

In [ ]:
_ = sns.pointplot(x="dose", y="len", data=data, hue='supp')
plt.show()

_ = sns.pointplot(x="supp", y="len", data=data, hue='dose')
plt.show()

Взаимодействие между факторами можно наблюдать, когда результат от воздействия одного фактора на зависимую переменную не одинаков при разных значениях другого фактора.

### Взаимодействие значимо, поэтому разделим выборку по способу приёма витамина.

In [ ]:
OJ = data[data['supp']=='OJ']
VC = data[data['supp']=='VC']
VC.shape, OJ.shape

#### Для морских свинок, принимающих аскорбиновую кислоту:

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=VC)
plt.show()

In [ ]:
lm = ols('len ~ C(dose)',data=VC).fit()
table = sm.stats.anova_lm(lm)
table

Для морских свинок, принимающих аскорбиновую кислоту влияние разных уровней фактора дозы неодинаково

##### Критерий Краскела-Уоллиса о равенстве средних всех групп

In [ ]:
st.kruskal(*VC.groupby('dose').groups.values())

##### Исследуем попарные различия в воздействии на целевой признак между всеми уровнями фактора дозы

In [ ]:
mc = MultiComparison(VC['len'], VC['dose'])
result = mc.tukeyhsd()
 
print(result)

Значимые различия во влиянии на целевую переменную есть между каждой парой групп

#### Для морских свинок, принимающих апельсиновый сок:

In [ ]:
_ = sns.boxplot(x='dose', y='len', data=OJ)
plt.show()

In [ ]:
lm = ols('len ~ C(dose)',data=OJ).fit()
table = sm.stats.anova_lm(lm)
table

Отвергаем гипотезу о том, что каждый уровень фактора дозы вносит одинаковый вклад в целевую переменную

##### Исследуем попарные различия в воздействии на целевой признак между всеми уровнями фактора дозы

In [ ]:
mc = MultiComparison(OJ['len'], OJ['dose'])
result = mc.tukeyhsd()
 
print(result)

Значимые отличия в воздействии на целевой признак отсутствуют только между 1 и 2

In [ ]:
_ = sns.pointplot(x="dose", y="len", data=OJ)
plt.title('Pointplot for orange juice')
plt.show()

### Итого

- для морских свинок, принимающих аскорбиновую кислоту, различия между размером дозы значимы всегда
- для морских свинок, принимающих апельсиновый сок, различия между размером дозы в 1 и 2 мг не значимо

## Марихуана и скорость реакции


Изучалось воздействие марихуаны на скорость реакции. 
В качестве испытуемых
были выбраны по 12 человек из каждой категории:
* никогда не пробовали марихуану;
* иногда употребляют марихуану;
* регулярно употребляют марихуану.

Испытуемые были разделены на две равные группы; половине из них дали
выкурить две сигареты с марихуаной, вторая половина выкурила две обычные
сигареты с запахом и вкусом марихуаны. Сразу после этого все испытуемые прошли
тест на скорость реакции.
Требуется оценить влияние марихуаны на скорость реакции, учитывая фактор
предыдущего опыта употребления.

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem6/data/weed.txt', delimiter='\t')
data.head()

In [ ]:
data.describe()

In [ ]:
placebo = data[data['Placebo/Marijuana']==1]
placebo.head()

In [ ]:
weed = data[data['Placebo/Marijuana']==2]
weed.head()

Переведем выборку в более адекватное представление: один объект == один человек

In [ ]:
new_data = []
for i in data.values:
    new_data.append({'placebo': i[0]==1, 'time': i[1], 'type':'none'})
    new_data.append({'placebo': i[0]==1, 'time': i[2], 'type':'light'})
    new_data.append({'placebo': i[0]==1, 'time': i[3], 'type':'moderate'})
new_data = pd.DataFrame(new_data)
new_data.head()

In [ ]:
len(new_data)

### Посмотрим распределение времени реакции

In [ ]:
sns.boxplot(data=new_data[new_data['placebo']==True], x='time', y='type' )
_ = plt.title('Placebo')
plt.show()

In [ ]:
sns.boxplot(data=new_data[new_data['placebo']==False], x='time', y='type' )
_ = plt.title('Weed')
plt.show()

### Дисперсионный анализ

#### Двухфакторный анализ (плацебо + тип курильщика)

In [ ]:
lm = ols('time ~ C(type) + C(placebo) + C(placebo):C(type)',  data=new_data).fit()

##### Что произошло?

$$X_{ijk} = \mu + \alpha_i + \beta_j + \gamma_{ij} + \varepsilon_{ijk},$$
$$i = 1, ..., K_1,\ j = 1, ..., K_2,\ k = 1, ..., n.$$

- $\mu$ -- общее среднее значение признака,
- $\alpha_i$ -- воздействие уровня $i$ фактора $f_1$,
- $\beta_j$ -- воздействие уровня $j$ фактора $f_2$,
- $\gamma_{ij}$ -- дополнительное воздействие комбинации уровней $i$ и $j$ факторов $f_1$ и $f_2$,
- $\varepsilon_{ijk}$ -- случайные независимые одинаково распределенные ошибки

- ```ols``` -- метод наименьших квадратов для задачи регрессии
- Внутри используется формула из ```patsy``` ([описание синтаксиса формул patsy](https://patsy.readthedocs.io/en/latest/formulas.html#formulas))

Краткий экскурс в формулы ```patsy```.

- Формула разделяется на:
    - левую часть: ```time``` -- целевой признак
    - правую часть: ```C(type) + C(placebo) + C(placebo):C(type)``` -- выражение, состоящее из факторов, чьё воздействие на целевой признак мы исследуем, и некоторых операций
- `~` -- разделение левой и правой части
- `+` -- берем во внимание только сами признаки, не учитывая их внутренние взаимодействия ($A \cup B \backslash A \cap B$)
- `:` -- берем во внимание только внутренние взаимодействия между признаками ($A \cap B$)
- `*` -- берем во внимание как сами признаки, так и их внутренние взаимодействия ($A \cup B$)
- `С(...)` -- категориальный признак

Можно посмотреть различную информацию о полученной модели

In [ ]:
lm.summary()

In [ ]:
sm.stats.anova_lm(lm)

In [ ]:
lm = ols('time ~ C(placebo)', data=new_data).fit()

In [ ]:
sm.stats.anova_lm(lm)

#### Критерий Стьюдента о равенстве матожиданий

В предположении о равенстве дисперсий

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], usevar='pooled')

Без предположения

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], usevar='unequal')

Левосторонняя альтернатива

In [ ]:
ttest_ind(new_data[new_data['placebo']==True]['time'], new_data[new_data['placebo']==False]['time'], alternative='smaller', usevar='pooled')

### Итог:

- факт того, принимал ли человек раньше марихуану, не влияет на его скорость реакции
- при выкуривании марихуаны реакция человека значительно зампедляется